In [1]:
import os
os.chdir("../")

from tqdm.notebook import tqdm, trange
from pyinstrument import Profiler
from joblib import Parallel, delayed

import semiolog as slg

semiotic = slg.Cenematic("fr_wiki",requested_cpu=4)

Vocabulary will not be loaded from file.



In [2]:
chain = list(" ".join(semiotic.corpus.train[:50000]).replace(" ",""))

In [30]:
from collections import Counter

pair_pos = Counter()
# pair_pos = {}

chain_zip = list(zip(chain,chain[1:]))

for i,k in tqdm(list(enumerate(chain_zip))):
    if k in pair_pos:
        pair_pos[k] += [i] #TODO: maybe there's a faster way to construct the values of this dict as sets
    else:
        pair_pos[k] = [i]

pair_pos = {k:set(v) for k,v in pair_pos.most_common()}

pair_len = Counter()
for k,pos in tqdm(pair_pos.items()):
    pair_len[k] = len(pos)


In [26]:
vocab_delta = 5
best_pair = pair_len.most_common(1)[0][0]
for _ in trange(vocab_delta):
    # print(best_pair)
    # print(pair_pos[best_pair])
    # profile = Profiler()
    # profile.start()
    for i in tqdm(pair_pos[best_pair], disable=False):
        # merge to the left
        left_pair_i = i-1
        while chain_zip[left_pair_i] == None:
            left_pair_i -= 1
        left_pair = chain_zip[left_pair_i]
        left_pair_pos = pair_pos[left_pair]
        # print(left_pair_i)
        # print(left_pair)
        # print(left_pair_pos)
        left_pair_pos.discard(left_pair_i)
        # print(left_pair_pos)
        new_pair = (left_pair[0],"".join(best_pair))
        if new_pair in pair_pos:
            pair_pos[new_pair] = pair_pos[new_pair].union({left_pair_i})
        else:
            pair_pos[new_pair] = {left_pair_i}
        pair_len[left_pair] -= 1
        pair_len[new_pair] += 1
        chain_zip[left_pair_i] = new_pair
        
        # merge to the right
        right_pair_i = i+1
        while chain_zip[right_pair_i] == None:
            right_pair_i += 1
        right_pair = chain_zip[right_pair_i]
        right_pair_pos = pair_pos[right_pair]
        # print(right_pair_i)
        # print(right_pair_pos)
        right_pair_pos.discard(right_pair_i)
        # print(right_pair_pos)
        new_pair = ("".join(best_pair), right_pair[1])
        if new_pair in pair_pos:
            pair_pos[new_pair] = pair_pos[new_pair].union({right_pair_i})
        else:
            pair_pos[new_pair] = {right_pair_i}
        
        pair_len[right_pair] -= 1
        pair_len[new_pair] += 1
        chain_zip[right_pair_i] = new_pair
        
        chain_zip[i] = None

             
    del pair_pos[best_pair]
    del pair_len[best_pair]
    best_pair = pair_len.most_common(1)[0][0]
    # profile.stop()
    # print(profile.output_text(unicode=True, color=True))



In [27]:
result_discard = pair_len

In [29]:
vocab_delta = 5
best_pair = pair_len.most_common(1)[0][0]
for _ in trange(vocab_delta):
    # print(best_pair)
    # print(pair_pos[best_pair])
    profile = Profiler()
    profile.start()
    
    for i in tqdm(pair_pos[best_pair], disable=False):
        # merge to the left
        left_pair_i = i-1
        while chain_zip[left_pair_i] == None:
            left_pair_i -= 1
        left_pair = chain_zip[left_pair_i]
        left_pair_pos = pair_pos[left_pair]
        # print(left_pair_i)
        # print(left_pair)
        # print(left_pair_pos)
        
        # left_pair_pos.discard(left_pair_i)
        
        # print(left_pair_pos)
        new_pair = (left_pair[0],"".join(best_pair))
        if new_pair in pair_pos:
            pair_pos[new_pair] = pair_pos[new_pair].add(left_pair_i)
        else:
            pair_pos[new_pair] = {left_pair_i}
        pair_len[left_pair] -= 1
        pair_len[new_pair] += 1
        chain_zip[left_pair_i] = new_pair
        
        # merge to the right
        right_pair_i = i+1
        while chain_zip[right_pair_i] == None:
            right_pair_i += 1
        right_pair = chain_zip[right_pair_i]
        right_pair_pos = pair_pos[right_pair]
        # print(right_pair_i)
        # print(right_pair_pos)
        
        # right_pair_pos.discard(right_pair_i)
        
        # print(right_pair_pos)
        new_pair = ("".join(best_pair), right_pair[1])
        if new_pair not in pair_pos:
            pair_pos[new_pair] = {right_pair_i}
            # print(new_pair in pair_pos)
            # print(new_pair)
            # print(set(pair_pos).intersection(set(new_pair)))
            # print(pair_pos[new_pair])
            
        else:
            pair_pos[new_pair] = pair_pos[new_pair].add(right_pair_i)
        
        pair_len[right_pair] -= 1
        pair_len[new_pair] += 1
        chain_zip[right_pair_i] = new_pair
        
        chain_zip[i] = None

             
    del pair_pos[best_pair]
    del pair_len[best_pair]
    best_pair = pair_len.most_common(1)[0][0]
    profile.stop()
    print(profile.output_text(unicode=True, color=True))

AttributeError: 'NoneType' object has no attribute 'add'

In [20]:
('ð', 'sqdqsdqsdqsds') in pair_pos

False

In [23]:
profile.stop()

In [38]:
pair_len.most_common()

[(('l', 'e'), 38592),
 (('e', 'r'), 37448),
 (('r', 'e'), 36293),
 (('t', 'i'), 35447),
 (('i', 's'), 32870),
 (('u', 'r'), 32552),
 (('a', 'r'), 32100),
 (('i', 'n'), 31684),
 (('l', 'a'), 31440),
 (('e', 't'), 29614),
 (('o', 'u'), 28775),
 (('t', 'e'), 28132),
 (('i', 't'), 26660),
 (('q', 'u'), 24764),
 (('a', 'i'), 24237),
 (('r', 'i'), 24234),
 (('a', 't'), 22768),
 (('en', 't'), 22749),
 (('t', 'r'), 22699),
 (('a', 'l'), 21910),
 (('e', 'l'), 21484),
 (('n', 'e'), 20757),
 (('o', 'r'), 20206),
 (('i', 'on'), 19633),
 (('r', 'a'), 19624),
 (('a', 'u'), 19400),
 (('u', 'n'), 19233),
 (('i', 'l'), 19076),
 (('s', 'e'), 18468),
 (('l', 'i'), 18414),
 (('e', 'm'), 18377),
 (('l', 'l'), 17956),
 (('t', 'a'), 17684),
 (('e', 'de'), 17616),
 (('e', 'u'), 17602),
 (('i', 'e'), 17580),
 (('l', 'es'), 17287),
 (('p', 'a'), 17162),
 (('u', 'e'), 16963),
 (('e', 'c'), 16397),
 (('r', 'o'), 15489),
 (('d', 'es'), 15340),
 (('s', 'i'), 15320),
 (('r', 't'), 14741),
 (('p', 'r'), 14721),
 (('u

In [35]:
def update_corpus(pair_pos,pair_len,chain_zip,i):
    left_pair_i = i-1
    while chain_zip[left_pair_i] == None:
        left_pair_i -= 1
    left_pair = chain_zip[left_pair_i]
    left_pair_pos = pair_pos[left_pair]
    # print(left_pair_i)
    # print(left_pair)
    # print(left_pair_pos)
    
    # left_pair_pos.discard(left_pair_i)
    
    # print(left_pair_pos)
    new_pair = (left_pair[0],"".join(best_pair))
    if new_pair not in pair_pos:
        pair_pos[new_pair] = {left_pair_i}        
    else:
        pair_pos[new_pair] = pair_pos[new_pair].union({left_pair_i})

    pair_len[left_pair] -= 1
    pair_len[new_pair] += 1
    chain_zip[left_pair_i] = new_pair
    
    # merge to the right
    right_pair_i = i+1
    while chain_zip[right_pair_i] == None:
        right_pair_i += 1
    right_pair = chain_zip[right_pair_i]
    right_pair_pos = pair_pos[right_pair]
    # print(right_pair_i)
    # print(right_pair_pos)
    
    # right_pair_pos.discard(right_pair_i)
    
    # print(right_pair_pos)
    new_pair = ("".join(best_pair), right_pair[1])
    if new_pair not in pair_pos:
        pair_pos[new_pair] = {right_pair_i}
        # print(new_pair in pair_pos)
        # print(new_pair)
        # print(set(pair_pos).intersection(set(new_pair)))
        # print(pair_pos[new_pair])
        
    else:
        pair_pos[new_pair] = pair_pos[new_pair].union({right_pair_i})
    
    pair_len[right_pair] -= 1
    pair_len[new_pair] += 1
    chain_zip[right_pair_i] = new_pair
    
    chain_zip[i] = None

vocab_delta = 5
best_pair = pair_len.most_common(1)[0][0]
for _ in trange(vocab_delta):
    # print(best_pair)
    # print(pair_pos[best_pair])
    # profile = Profiler()
    # profile.start()
    
    Parallel(n_jobs=4, prefer="threads")(delayed(update_corpus)(pair_pos,pair_len,chain_zip,i) for i in pair_pos[best_pair])
        
    del pair_pos[best_pair]
    del pair_len[best_pair]
    best_pair = pair_len.most_common(1)[0][0]

    # profile.stop()
    # print(profile.output_text(unicode=True, color=True))

In [23]:
result_discard == pair_len

False

In [39]:
for l in result_discard.keys():
    if result_discard[l] != pair_len[l]:
        print(l,result_discard[l], pair_len[l])

('—', 'r') 1 0
('<', 'l') 1 0
('s', 'Γ') 1 0
('e', 'ʻ') 2 0
('i', 'ā') 1 0
('e', 'Ⱘ') 1 0
('N', 'l') 2 1
('e', '°') 1 0
('s', '挑') 1 0
('/', 'l') 1 0
('r', 'Ö') 2 0
('s', 'ã') 3 1
('X', 'r') 1 0
('ț', 'e') 2 0
('Ş', 'e') 2 1
('e', 'á') 4 3
('·', 'l') 1 0
('r', 'å') 3 1
('Š', 't') 3 2
('ç', 'l') 2 0
('R', 'r') 3 2
('e', 'κ') 1 0
('s', '♕') 1 0
('ő', 'i') 1 0
('e', 'Š') 2 1
('i', '[') 4 3
('Ş', 't') 2 1
('ş', 't') 2 0
('e', 'ş') 2 1
('ō', 'r') 2 1
('r', 'Δ') 1 0
('L', 'r') 1 0
('Ś', 'l') 1 0
('[', 'r') 5 4
('e', 'ô') 2 1
('e', '♙') 2 1
('ř', 't') 1 0
('Y', 'l') 1 0
('æ', 'r') 5 2
('→', 'l') 2 0
('i', '<') 1 0
('e', '=') 5 3
('Ö', 'r') 4 2
('È', 'r') 1 0
('—', 'd') 3 2
('r', 'Î') 1 0
('ó', 't') 6 2
('e', 'å') 2 1
('P', 't') 8 7
('Q', 'e') 4 3
('ø', 'r') 3 2
('e', 'Î') 7 5
('e', '#') 2 1
('e', '−') 2 1
('D', 'l') 3 2
('́', 'l') 4 3
('?', 'l') 1 0
('e', 'č') 3 2
('e', 'Ž') 2 1
('ș', 't') 3 0
('e', 'ș') 6 3
(';', 't') 9 7
('ő', 'r') 4 3
('B', 't') 2 1
('9', 'i') 4 3
('i', 'č') 12 11
('v', 't

In [32]:
pair_len.most_common(10)

[(('l', 'e'), 38592),
 (('e', 'r'), 37448),
 (('r', 'e'), 36293),
 (('t', 'i'), 35447),
 (('i', 's'), 32870),
 (('u', 'r'), 32552),
 (('a', 'r'), 32100),
 (('i', 'n'), 31684),
 (('l', 'a'), 31440),
 (('e', 't'), 29614)]

In [33]:
result_discard.most_common(10)

[(('l', 'e'), 38592),
 (('e', 'r'), 37448),
 (('r', 'e'), 36293),
 (('t', 'i'), 35447),
 (('i', 's'), 32870),
 (('u', 'r'), 32552),
 (('a', 'r'), 32100),
 (('i', 'n'), 31684),
 (('l', 'a'), 31440),
 (('e', 't'), 29614)]